#Question Answering

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

In [ ]:
!ls
%cd drive/MyDrive/splore_project
!pwd
!ls

In [3]:
import pandas as pd
import numpy as np

###Reading dataset of merged conversations

In [52]:
dataset = pd.read_csv("merged_chats.csv")
dataset.drop(dataset[dataset['dialogue'].isnull()].index , inplace=True)
dataset

,Unnamed: 0,conv_id,dialogue
0,0,1,Are you a fan of Google or Microsoft? \n Both ...
1,1,2,do you like dance? \n Yes I do. Did you know ...
2,2,3,Hey what's up do use Google very often?I reall...
3,3,4,Hi! do you like to dance? \n I love to dance ...
4,4,5,do you like dance? \n I love it. Did you know ...
...,...,...,...
95,95,96,"Dogs or cats? \n Hi! I have two cats, but wan..."
96,96,97,Crazy that early humans used to have to battle...
97,97,98,"Hi, Do you like to cook? \n I love to cook. j..."
98,98,99,Hi do you like tennis? \n Oh yeah! I pick up a...


###Calculating start & end offsets for each conversation for further use

In [53]:
offsets1 = []
offsets2 = []
size = 0
for x in range(len(dataset)):
  offsets1.append(size)
  size += len(dataset.dialogue[x])
  offsets2.append(size)

###Storing the offsets

In [54]:
dataset['start_offset'] = offsets1
dataset['end_offset'] = offsets2
dataset

,Unnamed: 0,conv_id,dialogue,start_offset,end_offset
0,0,1,Are you a fan of Google or Microsoft? \n Both ...,0,2010
1,1,2,do you like dance? \n Yes I do. Did you know ...,2010,3130
2,2,3,Hey what's up do use Google very often?I reall...,3130,5150
3,3,4,Hi! do you like to dance? \n I love to dance ...,5150,6915
4,4,5,do you like dance? \n I love it. Did you know ...,6915,8138
...,...,...,...,...,...
95,95,96,"Dogs or cats? \n Hi! I have two cats, but wan...",214916,216672
96,96,97,Crazy that early humans used to have to battle...,216672,220308
97,97,98,"Hi, Do you like to cook? \n I love to cook. j...",220308,223009
98,98,99,Hi do you like tennis? \n Oh yeah! I pick up a...,223009,225566


In [ ]:
!pip install transformers huggingface_hub
!pip install accelerate -U
!pip install datasets

In [9]:
import transformers
from transformers import pipeline
import torch, gc
import torch.nn as nn

###Using inference pipeline for Roberta model pre-trained on Squad2 dataset on QA task

In [ ]:
generator = pipeline(model='deepset/roberta-base-squad2')

###Set of 5 sample questions

In [62]:
questions = ["Who thinks aids is a manmade disease?",
             "Which fish has a neck?",
             "What type of dancer is Bruce Lee?",
             "Where is google plex?",
             "How do cats hear?"
            ]

test_context = ' \n '.join(dataset.dialogue[:20])  ## taking first 20 conversations for faster inference

###Using start & end locations of the answers returned by the model to get conversation_ids via newly created offset columns

In [64]:
for q in questions:
  gen_op = generator(question=q, context=test_context)
  print(gen_op)
  c_id = dataset[(dataset['start_offset'] <= gen_op['start']) & (gen_op['end'] <= dataset['end_offset'])].conv_id.item()
  print("conv_id: "+str(c_id))
  print("\n\n")

{'score': 0.7361200451850891, 'start': 40763, 'end': 40773, 'answer': 'Kanye West'}
conv_id: 19



{'score': 0.9227142930030823, 'start': 23100, 'end': 23109, 'answer': 'seahorses'}
conv_id: 12



{'score': 0.42452365159988403, 'start': 2074, 'end': 2081, 'answer': 'cha cha'}
conv_id: 2



{'score': 0.9199182987213135, 'start': 3375, 'end': 3399, 'answer': 'Mountain View California'}
conv_id: 3



{'score': 0.11570027470588684, 'start': 1882, 'end': 1933, 'answer': 'too faint or too high frequency human ears can hear'}
conv_id: 1



